In [5]:
import pandas as pd
import os 
import pathlib 
import glob
from datetime import datetime
from tqdm import tqdm

from dotenv import load_dotenv
from elasticsearch import Elasticsearch

import minio
from minio.error import S3Error
from minio.commonconfig import ENABLED
from minio.versioningconfig import VersioningConfig

from RDSBucket_class import *
from data_profiles import *

import warnings
warnings.filterwarnings("ignore")

#####----------------------------------------------------------------#####
##### preprocessing metadata for bam file
#####----------------------------------------------------------------#####

path_to_save_prep_metadata = "/Users/hieunguyen/src/DVC_system/examples/dummy_from_real/prep_metadata"
os.system("mkdir -p {}".format(path_to_save_prep_metadata))

path_to_main_input = "./examples/dummy_data"    
minio_credential = "credentials.mb.json"
es_credential = "es_credential.json"

donwloaddir = "./examples/download"
os.system("mkdir  -p {}".format(donwloaddir))


es = ESearch(es_credential = es_credential)

search_query = {
  "query": {
    "match": {
      "Labcode": "ZLBE113NB"
    }
  }
}

response = es.es.search(index=["wgsfeature", "wgsbam"], body=search_query, size = 1000)

search_resdf = pd.DataFrame([doc['_source'] for doc in response['hits']['hits']])

In [9]:
search_resdf[["FileName", "versionID", "bucket"]]

,FileName,versionID,bucket
0,9-ZLBE113NB_S95053-S97053_GWfeature_CNA_short.rds,768529ae-e421-4d9b-a2ba-f78e8176870a,wgsfeatures
1,9-ZLBE113NB_S95053-S97053_GWfeature_CNA.rds,126d0b3d-489b-48b9-8d2f-5ec315563e5c,wgsfeatures
2,9-ZLBE113NB_S95053-S97053_GWfeature_CNA_short.csv,f1873166-4df8-447c-b725-e5d0684099bc,wgsfeatures
3,9-ZLBE113NB_S95053-S97053_GWfeature_EM_for_eac...,1ab93ea5-0b2b-4740-9db8-2cb66361ac13,wgsfeatures
4,9-ZLBE113NB_S95053-S97053_GWfeature_flen_ratio...,6eb91c98-3182-4f1b-a129-9b7be04b53b2,wgsfeatures
5,9-ZLBE113NB_S95053-S97053_GWfeature_EM.csv,0a7f0008-1ab7-46f7-8c2b-a13665e30a8c,wgsfeatures
6,9-ZLBE113NB_S95053-S97053_GWfeature_CNA.csv,b7183d56-734a-4383-bb62-5ae5ee1d8145,wgsfeatures
7,9-ZLBE113NB_S95053-S97053_GWfeature_Nucleosome...,b421a6dd-c048-41a7-a23e-64fc85c11a6a,wgsfeatures
8,9-ZLBE113NB_S95053-S97053_GWfeature_flen_for_e...,89500194-a709-42a3-bc58-ca7f2dbfb216,wgsfeatures
9,9-ZLBE113NB_S95053-S97053_GWfeature_readCounts...,18678da7-fe3c-4a7b-bb4b-566dceaa0663,wgsfeatures
